In [1]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../../adult-data"        

In [2]:
# (fold cell) Load and prep web browsing data
# output = df with visits to adult sites
# usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols_web_desktop = ["private_domain"]
usecols = ["private_domain"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
        )
    ])
     )
print("output = df with web browses")
print(f"{len(df)=:,}")
df.head()

output = df with web browses
len(df)=6,297,382


,private_domain
0,google.com
1,coupons.com
2,google.com
3,coupons.com
4,google.com


In [3]:
df_domains_ranked = (
    df.reset_index()
    .groupby("private_domain").size().reset_index()
    .sort_values(0, ascending=False, ignore_index=True)
    .rename_column(0, "tt_visits")
)
df_domains_ranked

,private_domain,tt_visits
0,google.com,986722
1,facebook.com,449173
2,yahoo.com,245982
3,bing.com,235769
4,youtube.com,232829
...,...,...
64069,birdwatchingtips.com,1
64070,consultanthelpers.com,1
64071,optyruntchan.com,1
64072,oracionesmilagrosasypoderosas.com,1


In [4]:
def split_dataframe(dataframe: pd.DataFrame, n_chunks: int):
    """Split a dataframe into n chunks of approximately equal size."""
    
    # calculate the approximate chunk size
    chunk_size = len(dataframe) // n_chunks
    
    # split the dataframe into chunks
    chunks = [dataframe[i:i+chunk_size] for i in range(0, len(dataframe), chunk_size)]
    
    if len(chunks) > n_chunks:  # More chunks than specified, b/c of remainder rows
        # Concatenate the last two chunks
        chunks = chunks[:-2] + [pd.concat([chunks[-2], chunks[-1]])]
    return chunks

In [5]:
for ix, chunk in enumerate(split_dataframe(df_domains_ranked.select_columns("private_domain"), n_chunks=8)):
    (chunk
     .assign(return_code=np.nan)
     .to_csv(f"chunk{ix}.csv", index=False)
    )

In [6]:
!ls -lh

total 3.0M
-rw-r--r-- 1 lsys lsys 124K Apr 28 16:44 chunk0.csv
-rw-r--r-- 1 lsys lsys 130K Apr 28 16:44 chunk1.csv
-rw-r--r-- 1 lsys lsys 133K Apr 28 16:44 chunk2.csv
-rw-r--r-- 1 lsys lsys 138K Apr 28 16:44 chunk3.csv
-rw-r--r-- 1 lsys lsys 137K Apr 28 16:44 chunk4.csv
-rw-r--r-- 1 lsys lsys 138K Apr 28 16:44 chunk5.csv
-rw-r--r-- 1 lsys lsys 139K Apr 28 16:44 chunk6.csv
-rw-r--r-- 1 lsys lsys 137K Apr 28 16:44 chunk7.csv
-rw-r--r-- 1 lsys lsys 1.8M Apr 27 19:43 domains_categories.csv
-rw-r--r-- 1 lsys lsys  37K Apr 27 23:22 google.com.json
-rw-r--r-- 1 lsys lsys  11K Apr 28 16:44 rank-chunk.ipynb
-rw-r--r-- 1 lsys lsys  21K Apr 28 16:11 test-vt-query-google-domain.ipynb
-rw-r--r-- 1 lsys lsys  20K Apr 27 19:43 yougov_domain_categories.ipynb
